In [ ]:
7:35 pm Análisis de cohortes
1. Buscar cuándo fue la primera visita de cada usuario
   primeras_visitas = visitas.groupby('uid')['start_ts'].min().reset_index()
   primeras_visitas.columns=['uid','inicio_ts_primera_sesion']
   primeras_visitas['primera_sesion_dia']=primeras_visitas['inicio_ts_primera_sesion'].dt.date
   primeras_visitas['primera_sesion_mes']=primeras_visitas['inicio_ts_primera_sesion'].dt.month
2. Tabla de visitas completa
   visitas_full = pd.merge(visitas_raw,primeras_visitas,how='inner',on='uid')
3. Calcular la edad que ha pasado desde la primera visita hasta la visita actual (primera visita contra fecha actual de visita)
4. Hay que obtener el primer día de cada mes de la columna inicio_ts_primera_sesion
    primeras_visitas['primera_sesion_mes'] = primeras_visitas['inicio_ts_primera_sesion'].apply(lambda fecha: pd.to_datetime(fecha.strftime('%Y-%m-01')))
5. Calcular la edad en meses
   visitas_full['edad_meses'] = (visitas_full['sesion_mes'] - visitas_full['primera_sesion_mes'])
6. Calcular el número de visitas de cada usuario en el tiempo
    visitas_full['edad_meses'] = ((visitas_full['sesion_mes'] - visitas_full['primera_sesion_mes']) / (30 * np.timedelta64(1,'D'))).round().astype('int64')
7. Crear las cohortes
    cohortes = visitas_full.pivot_table(
                            index='primera_sesion_mes',
                            columns='edad_meses',
                            values='uid',
                            aggfunc='nunique')
 cohortes.fillna('')
****** Cantidad de visitantes es una métrica. La retención es un KPI
8. Calcular la retención
    retencion = pd.DataFrame() #Dataframe vacío
    for edad_mes in cohortes.columns:
        retencion = pd.concat([retencion,cohortes[edad_mes]/cohortes[0],axis=1)
    retencion.columns = cohortes.columns
    retencion.index = [str(fecha)[:10]for fecha in retencion.index]
    retencion
9. Graficas
    plt.figure(figsize=(13,9))
    sns.heatmap(retencion,annot=True,fmt='.1%',linewidht=1,linecolor='grey',vmax=0.1,cbar_kws={'orientation':'horizontal'})

CUANDO EMPIEZA LA GENTE A COMPRAR

In [ ]:
ordenes_raw.head()
ordenes_raw['dia_compra'] = ordenes_raw['buy_ts'].dt.date
ordenes_raw['mes_compra'] = ordenes_raw['buy_ts'].apply(lambda fecha: pd.to_datetime(fecha.strftime('%Y-%m-01')))
primeras_compras = ordenes_raw.groupby('uid')['mes_compra'].min().reset_index()
primeras_compras.columns=['uid','primera_compra_mes']
primeras_compras['primera_compra_dia'] = primeras_compras['primera_compra_ts'].dt.date
primeras_compras['primera_compra_mes'] = primeras_compras['primera_compra_dia'].apply(lambda fecha: pd.to_datetime(fecha.strftime('%Y-%m-01')))
compradores = pd.merge(primeras_visitas,primeras_compras,how='inner',on='uid')
compradores['dias_a_la_primera_compra'] =  (compradores['primera_compra_ts']-comrpadores['inicio_ts_primera_sesion']) / np.timedelta64(1,'D').astype('int')
compradores['dias_a_la_primera_compra'].plot(kind='hist',bins=100,figsize=(12,7)).set(title='Dias desde la primera visita a la compra',xlabel='Dias',ylabel='frecuencia')
plt.show()

Obtener el tamaño de las cohortes
tamano_cohortes = compradores.groupby('primera_compra_mes')['uid'].nunique().reset_index() #Con cuantos compradores inición cada cohorte
tamano_cohortes.columns=['primera_compra_mes','total_compradores']

Obtener las cohortes
cohortes = pd.merge(ordenes_raw,compradores,how='inner',on='uid').groupby(['primera_compra_mes','orden_mes']).agg({'revenue':'count'}).reset_index() #SI AL REVENUE SE LE HACE SUM SE OBTIENE EL TOTAL DE DINERO POR COHORTE
#LTV. Número de órdenes por usuario. Costo promedio de las órdenes. 
cohortes['edad_mes'] = ((cohortes['mes_compra']-cohortes['primera_compra_mes']) / (30 *np.timedelta64(1,'D'))).round().astype('int')
cohortes.columns=['primer_compra_mes','mes_compra','n_ordenes','edad_mes']
cohortes_reporte = pd.merge(tamano_cohortes,cohortes,how='inner',on='primera_compra_mes')

#Metrica de ordenes por comprador
cohortes_reporte['ordenes_por_comprador'] = cohortes_reporte['num_ordenes']/cohortes_reporte['total_compradores']

#Tabla pivote para LTV
cohortes_ltv = cohortes_reporte.pivot_table(
                                index='primera_compra_mes',
                                columns='edad_mes',
                                values='ordenes_por_comprador',
                                aggfunc='sum'
) #Muestra el número de órdenes por comprador a través del tiempo

cohortes_ltv = cohortes_reporte.pivot_table(
                                index='primera_compra_mes',
                                columns='edad_mes',
                                values='ordenes_por_comprador',
                                aggfunc='sum'
).cumsum() #Muestra el número de órdenes por comprador acumulada a través del tiempo

cohortes_ltv.round(2).fillna(0)
cohortes_ltv.index = cohortes_ltv.index.astype(str)
plt.figure(figsize=(13,9))
sns.heatmap(cohortes_ltv,annot=True,fmt='.2f',linewidth=1,linecolor='grey',cbar_kws={'orientation':'horizontal'})
plt.show()

In [ ]:
#Determinar la primera compra de cada usuario único
first_buy = clean_orders.groupby('uid')['buy_ts'].min()
#Cambio el nombre de la columna
first_buy.name = 'first_buy_date'
print(first_buy)

#Crear un nuevo dataframe para este análisis
first_buy_df = clean_visits[['uid','source_id','first_visit_date']].copy()

#Uno ambas tablas por la izquierda buscando por uid
first_buy_df = first_buy_df.join(first_buy,on='uid')
display(first_buy_df)

#Elimino registros con valores ausentes en la columna first_buy_date
first_buy_df = first_buy_df.dropna(subset=['first_buy_date'])
display(first_buy_df)

#Calculo el número de días trascurridos entre la primera visita y la primera compra
first_buy_df['conversion_time'] = (first_buy_df['first_buy_date'] - first_buy_df['first_visit_date']).dt.days

display(first_buy_df)
display(first_buy_df.dtypes)

def categorize(days):
    if days == 0:
        return '0 days'
    elif days == 1:
        return '1 day'
    elif days >= 2 and days <=60:
        return '(2-60) days'    
    elif days >= 61 and days <=10:
        return '(61-120) days'    
    elif days >= 121 and days <=180:
        return '(121-180) days'    
    elif days >= 181 and days <=240:
        return '(181-210) days'    
    elif days >= 241 and days <=300:
        return '(241-300) days'    
    else:
        return '(301-364) days'

# Aplicar la función a la columna
first_buy_df['conversion_category'] = first_buy_df['conversion_time'].apply(categorize)

display(first_buy_df)

# first_buy_df['first_buy_day'] = first_buy_df['first_buy_date'].dt.date
# first_buy_df['first_buy_month'] = first_buy_df['first_buy_date'].dt.month
# first_buy_df['first_visit_month'] = first_buy_df['first_visit_date'].dt.month
# display(first_buy_df)
# first_buy_df.loc[mask,'conversion_time_months'] = (first_buy_df.loc[mask,'first_buy_month'] - first_buy_df.loc[mask,'first_visit_month']).astype('Int64')
# display(first_buy_df)
# display(first_buy_df.dtypes)

In [ ]:
conversion_distribution = first_buy_df['conversion_category'].value_counts().reset_index()
conversion_distribution.columns = ['conversion_category','count']
display(conversion_distribution)
# cohort_grouped = first_buy_df.groupby('conversion_time').agg({'uid':'nunique'}).reset_index()
# display(cohort_grouped)
# cohort_grouped.hist(column='conversion_time',bins=50)

pedidos hechos durante un periodo de tiempo

In [ ]:
#Copiar el dataframe con fechas de primera visita y primera compra.
visits_orders = pd.merge(clean_visits,clean_orders,on='uid',how='left')
visits_orders_clean = visits_orders.loc[:, ['uid','end_ts','start_ts','buy_ts']]
# display(visits_orders_clean)
visits_orders_notna = visits_orders_clean.copy()
visits_orders_notna = visits_orders_notna.dropna(subset=['buy_ts']).reset_index(drop=True)
display(visits_orders_notna)

In [ ]:
visits_orders_schedule = visits_orders_notna.copy()

first_buy = visits_orders_schedule.groupby('uid')['buy_ts'].min().reset_index()
first_buy.columns = ['uid','first_buy']
display(first_buy)

In [ ]:
buy_dates = pd.merge(visits_orders_schedule,first_buy,on='uid',how='left')
display(buy_dates)

In [ ]:
buys_per_month = buy_dates.copy()
buys_per_month['buy_ts'] = pd.to_datetime(buys_per_month['buy_ts'])
buys_per_month['first_buy'] = pd.to_datetime(buys_per_month['first_buy'])
buys_per_month['buy_ts_month'] = buys_per_month['buy_ts'].dt.to_period('M')
buys_per_month['first_buy_month'] = buys_per_month['first_buy'].dt.to_period('M')
display(buys_per_month)

In [ ]:
buys_cohort = buys_per_month.copy()

buys_cohort['lifetime'] = (buys_cohort['buy_ts_month'] - buys_cohort['first_buy_month']).apply(lambda x: x.n)

display(buys_cohort)

#Número de pedidos por mes
orders_cohort = buys_cohort.groupby(['first_buy_month','lifetime']).agg({'uid':'count'}).reset_index()
display(orders_cohort)

In [ ]:
orders_cohort.rename(columns={'uid':'orders_count'},inplace=True)
display(orders_cohort)

In [ ]:
#Tabla dinámica para cohortes
orders_cohort_pivot = orders_cohort.pivot_table(index='first_buy_month',
                                                columns='lifetime',
                                                values='orders_count',
                                                aggfunc='sum')
display(orders_cohort_pivot.fillna('-'))